In [ ]:
import time

from Attestation import Attestation
from algebra import *
from fast_stark import FastStark
from ip import ProofStream
def load_trace_from_file(path):
    execution_path = {}
    with open(path, 'r') as file:
        lines = file.readlines()
        # Create a list to store the content
        # Iterate through each line
        start = True
        for line in lines:
            # Remove the newline character and split by comma
            parts = line.strip().split(' ')
            if start:
                start = False
                split_list= parts[0].strip().split('=')
                execution_path["start"] = FieldElement(int(split_list[1]), Field.main())
                split_list= parts[1].strip().split('=')
                execution_path["end"] = FieldElement(int(split_list[1]), Field.main())
                start_node = {}
                start_node["type"] = "start"
                start_node["dest"] = execution_path["start"]
                start_node["return"] = execution_path["start"]
                execution_path["path"] = [start_node]
                continue
            # select the second element
            if "call" in parts:
                jmp = {}
                jmp["type"] = "call"
                jmp["dest"] = FieldElement(int(parts[1]), Field.main())
                jmp["return"] = FieldElement(int(parts[2]), Field.main())
                execution_path["path"].append(jmp)
            elif "ret" in parts:
                jmp = {}
                jmp["type"] = "ret"
                jmp["dest"] = FieldElement(int(parts[1]), Field.main())
                jmp["return"] = FieldElement(int(parts[1]), Field.main())
                execution_path["path"].append(jmp)

            else:
                jmp = {}
                jmp["type"] = "jmp"
                jmp["dest"] = FieldElement(int(parts[1]), Field.main())
                jmp["return"] = FieldElement(int(parts[1]), Field.main())
                execution_path["path"].append(jmp)
            # Append the second element to the content list
    return execution_path
def load_cfg(path):
    cfg = {}
    with open(path, 'r') as file:
        lines = file.readlines()
        # Create a list to store the content
        # Iterate through each line
        for line in lines:
            # Remove the newline character and split by comma
            parts = line.strip().split(' ')
            # select the first element
            src = int(parts[0])
            dests = [int(dest) for dest in parts[1:]]
            cfg[src] = dests
        return cfg



In [ ]:
cfg = {0: [1, 2], 1: [3], 2: [4], 3: [5], 4: [], 5:[]} # Example CFG

#        +-----+
#        |  0  | jmp 1, jmp 2
#        +-----+
#         /   \
#        v     v
#     +-----+ +-----+
#     |  1  | |  2  | call 3 5, jmp 4
#     +-----+ +-----+
#        |       |
#        v       v
#     +-----+ +-----+
#     |  3  | |  4  | ret 5
#     +-----+ +-----+
#        |
#        v
#     +-----+
#     |  5  |
#     +-----+#
# Execution path: # start -> 0 -> 1 -> 3 -> 5

path = "example_trace.txt"
execution_path = load_trace_from_file(path)

a = Attestation(cfg)
execution = load_trace_from_file(path)


In [ ]:
nonce = FieldElement(100, Field.main())
start = FieldElement(0, Field.main())
end = FieldElement(5, Field.main())

start = time.time()
state = a.trace(nonce, start, end, execution["path"])
boundary = a.boundary_constraints(nonce,start,end)
stark = FastStark(Field.main(), 4, 2, 2, a.num_registers, a.num_cycles, transition_constraints_degree=a.max_adjacency+1)
air  = a.transition_constraints(stark.omicron)
transition_zerofier, transition_zerofier_codeword, transition_zerofier_root = stark.preprocess()

end = time.time()
print("Execution time setup: ", end - start)
start = time.time()
proof = stark.prove(state, air, boundary, transition_zerofier, transition_zerofier_codeword)
end = time.time()
verdict = stark.verify(proof, air, boundary, transition_zerofier_root)
print(verdict)
# print("Execution time: ", end - start)
print("Execution time: ", end - start)
#Size of the proof
print("Size of the proof: ", len(proof))
